In [1]:
from openai import OpenAI
import os

client = OpenAI(
    base_url='https://openrouter.ai/api/v1',
    api_key=os.environ.get('OPENROUTER_KEY')
)

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


In [6]:
NEW_PHRASE_PROMPT = \
"""You are correcting ARPAbet transcriptions of a word that were produced by an automated system.
You will be given two items, enclosed in curly braces:
1. The word, as written in English (this word may be misspelled, or may be repetitions of another phrase).
2. The current, low-quality ARPAbet transcription of the word, as produced by an automated
system. This system did not include vowel stress numbers in its output.
The transcription may have significant errors, such as missing or swapped syllables.

When correcting, there are two factors you need to consider:
- You must add correct stress number markings to any ARPAbet vowels.
- You must fix any faulty pronunciation transcriptions.

Your will produce outputs in JSON format for this word with the following keys:
1. 'transcription': A corrected, accurate ARPAbet transcription.
2. 'confidence': Your level of confidence in this transcription (high, medium, low). This should be based on whether the word is familiar to you.
Even if you believe the word is misspelled or a repeated phrase, assume that the word is correctly spelled and make your best attempt at pronouncing according to its spelling verbatim, including repetitions.
Do not transfer any punctuation marks, including single quotes or hyphens, to the ARPAbet transcription.

For example, given the following input: {discord's}, {CH R IH S K AO R D S}, the correct output ('thinking' omitted) is:
{
    'transcription': 'D IH1 S K AO0 R D S',
    'confidence': 'high'
}

Here is the input: {[[baseword]]}, {[[transcription]]}
Provide only your output:
"""

NEW_PHRASE_THINKING_PROMPT = \
"""You are correcting ARPAbet transcriptions of a word that were produced by an automated system.
You will be given two items, enclosed in curly braces:
1. The word, as written in English (this word may be misspelled, or may be repetitions of another phrase).
2. The current, low-quality ARPAbet transcription of the word, as produced by an automated
system. This system did not include vowel stress numbers in its output.
The transcription may have significant errors, such as missing or swapped syllables.

When correcting, there are two factors you need to consider:
- You must add correct stress number markings to any ARPAbet vowels.
- You must fix any faulty pronunciation transcriptions.

Your will produce outputs in JSON format for this word with the following keys:
1. 'thinking': Your thinking process, considering both vowel stress and faulty/correct pronunciation.
2. 'transcription': A corrected, accurate ARPAbet transcription.
3. 'confidence': Your level of confidence in this transcription (high, medium, low). This should be based on whether the word is familiar to you.
Even if you believe the word is misspelled or a repeated phrase, assume that the word is correctly spelled and make your best attempt at pronouncing according to its spelling verbatim, including repetitions.
Do not transfer any punctuation marks, including single quotes or hyphens, to the ARPAbet transcription.

For example, given the following input: {discord's}, {CH R IH S K AO R D S}, a correct output is:
{
    'thinking': 'Your thinking here',
    'transcription': 'D IH1 S K AO0 R D S',
    'confidence': 'high'
}

Here is the input: {[[baseword]]}, {[[transcription]]}
Provide only your output:
"""

In [8]:
test_cases = [
    #('chimerae', 'IH M EH R AH'),
    ("pony's", 'P OW N IY S'),
    #('OMIGOSHOMIGOSHOMIGOSHOMIGOSH', 'OW M AY G AA SH OW M AY G AA SH OW M AY G AA SH OW M AY G AA SH'),
    #('bufogren', 'B UW F OW G R EH N'),
    #('chicomoztoc', 'IY K OW UH'),
]

for case in test_cases:
    baseword, transcription = case
    prompt = NEW_PHRASE_THINKING_PROMPT.replace('[[baseword]]', baseword)
    prompt = prompt.replace('[[transcription]]', transcription)
    completion = client.chat.completions.create(
        model='google/gemini-flash-1.5',
        messages=[
            {'role': 'user',
            'content': prompt}
        ]
    )
    print(completion.usage)
    print(completion.choices[0].message.content)

CompletionUsage(completion_tokens=68, prompt_tokens=386, total_tokens=454)
{
    'thinking': 'The word is "pony's", which is a possessive form of "pony". The transcription should reflect the pronunciation of the word with the stress on the first syllable.',
    'transcription': 'P OW1 N IY0 Z',
    'confidence': 'high'
} 



In [11]:
import json
from tqdm import tqdm
from json_repair import repair_json

output_json = 'horsewords_raw_output_thinking.json'
output_data = {}

i = 1
with open('horsewords.clean') as f:
    while line := f.readline():
        spl = line.split('  ')
        baseword, transcription = spl
        baseword = baseword.lower()
        print("processing baseword "+baseword+f' ({i})')
        prompt = NEW_PHRASE_THINKING_PROMPT.replace('[[baseword]]', baseword)
        prompt = prompt.replace('[[transcription]]', transcription)
        completion = client.chat.completions.create(
            model='google/gemini-flash-1.5',
            messages=[
                {'role': 'user',
                'content': prompt}
            ]
        )
        content = completion.choices[0].message.content
        j_content = repair_json(content, return_objects=True)
        if len(j_content):
            output_data[baseword] = j_content
        else:
            output_data[baseword] = {'error': True, 'content': j_content}

        with open(output_json, 'w') as of:
            json.dump(output_data, of)
        i += 1


processing baseword pony's (1)
processing baseword spell's (2)
processing baseword friendship's (3)
processing baseword griffons' (4)
processing baseword tartarus (5)
processing baseword i-it's (6)
processing baseword hoarder (7)
processing baseword ember's (8)
processing baseword enchantress (9)
processing baseword sunburst's (10)
processing baseword foals (11)
processing baseword pie's (12)
processing baseword unicorns (13)
processing baseword unicorn's (14)
processing baseword canis (15)
processing baseword tenochtitlan (16)
processing baseword spike's (17)
processing baseword hippogriff (18)
processing baseword hippogriffs (19)
processing baseword cutie (20)
processing baseword trixie's (21)
processing baseword i-i (22)
processing baseword princesses' (23)
processing baseword shh (24)
processing baseword discord's (25)
processing baseword super-duper (26)
processing baseword bestest (27)
processing baseword sunset's (28)
processing baseword armor's (29)
processing baseword flitter 